### Приведение изображений к единому виду

Данный код предназначен для автоматического обрезания и изменения размера изображений до квадратной формы с заданной длиной стороны (по умолчанию 720 пикселей). 

Основная функция `crop_and_resize` принимает путь к изображению, опционально путь для сохранения результата и желаемый размер стороны. 
Сначала изображение считывается с диска, затем из него вырезается центральная квадратная область (по меньшей стороне), после чего эта область масштабируется до нужного размера. 
Если указан путь для сохранения, результат сохраняется на диск.

Применение:
- Подготовка датасетов для машинного обучения, где требуется одинаковый размер и форма изображений (например, для нейросетей).
- Приведение фотографий к единому виду для публикации в социальных сетях, галереях, аватарках и т.д.
- Быстрая обработка больших коллекций изображений для визуализации или анализа.

Преимущества:
- Автоматизация рутинной задачи по приведению изображений к единому формату.
- Сохранение центральной части изображения, что часто важно для портретов и объектов в центре кадра.
- Простота использования и интеграции в другие проекты.

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
def crop_and_resize(img_path, 
                    out_path=None, 
                    size=720):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"No such file: {img_path}")
        return
    h, w = img.shape[:2]
    
    min_side = min(h, w)
    start_x = (w - min_side) // 2
    start_y = (h - min_side) // 2
    cropped = img[start_y:start_y+min_side, 
                  start_x:start_x+min_side]
    
    resized = cv2.resize(cropped, (size, size))
    if out_path is not None:
        cv2.imwrite(str(out_path), resized)

    return resized

In [ ]:
axiom_data_path = Path("../data/axiom_data")
axiom_images = os.listdir(axiom_data_path)

img_path = axiom_data_path / axiom_images[0]
res = crop_and_resize(img_path, img_path)
plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

### Работа с видео

Данный код предназначен для работы с видеофайлами. 

Он позволяет:
- извлекать кадры из видео с определённым шагом (например, каждый 30-й кадр)
- обрезать их до квадратной области по центру и изменять размер до заданного значения (по умолчанию 720 пикселей). 

Полученные изображения сохраняются в указанную папку. Это удобно для подготовки датасетов, анализа видеоматериалов или создания подборок изображений из видео.

In [ ]:
def extract_frames_from_video(video_path, 
                              out_dir, 
                              step=30, 
                              size=720):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    cap = cv2.VideoCapture(str(video_path))
    frame_idx = 0
    saved_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % step == 0:
            h, w = frame.shape[:2]
            min_side = min(h, w)
            start_x = (w - min_side) // 2
            start_y = (h - min_side) // 2
            cropped = frame[start_y:start_y+min_side, 
                            start_x:start_x+min_side]
            resized = cv2.resize(cropped, (size, size))
            out_path = out_dir / f"{saved_idx:06d}.jpg"
            cv2.imwrite(str(out_path), resized)
            saved_idx += 1
        frame_idx += 1
    cap.release()
    print(f"Save {saved_idx} frames to {out_dir}")

In [ ]:
extract_frames_from_video("path/to/video.mp4", "output_folder", step=30)